<h1><center>Laboratorio 6: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Rodrigo Mella S.
- Nombre de alumno 2: Miguel Morales M.


### **Link de repositorio de GitHub:** `https://github.com/migmoralesmar/MDS7202/tree/Laboratorio6`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- Fecha de entrega: 09/06/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
# !pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la farbicación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [2]:
df_retail = pd.read_pickle("online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.
- **Recency (R)**: Indica la actualidad de la interacción de un cliente con la empresa, y da información sobre la tendencia a repetir la compra. Se define como: 
$$Recency(n)=\dfrac{1}{n} \sum^n_{i=1} date\_diff(t_{fecha final}, t_{m-i+1})$$

 Donde $date\_diff$ representa la diferencia en días entre la fecha de finalización del periodo de observación ($t_{fecha final}$), y la fecha de una visita del cliente cercana a $t_{fecha final}$, $t_{m-i+1}; t_{m}$ es la última visita del cliente; y n es el número de visitas recientes del cliente consideradas.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**Nota:** Para la $fechafinal$ utilice la fecha máxima del dataset más 1 día.

**Ejemplo de Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [3]:
def custom_features(dataframe):
    dataframe_in = dataframe.copy()
    dataframe_in['Gains'] = dataframe_in['Quantity']*dataframe_in['Price']
    dataframe_in['Date'] = pd.to_datetime(dataframe_in['InvoiceDate']).dt.date
    
    last_date = dataframe_in['Date'].max() + datetime.timedelta(days=1)
    
    # Dataframe auxiliares para extraer valores de fecha maxima y minima, y conteo de Invoice
    data_in = dataframe_in.copy().drop_duplicates(subset=['Invoice','Customer ID'])
    info = data_in.groupby(by=['Customer ID']).agg({'Date': ['min', 'max'],'Invoice': ['count']}).reset_index()
    
    # Se utiliza n = 5 para estimar Recency
    n = 5
    info2 = data_in.groupby(by=['Customer ID'],as_index=False).agg({'Date':lambda x: (last_date - x).
                    sort_values(ascending=False).drop_duplicates()[0:n].sum().days/n})
    
    
    info3 = data_in.groupby(by=['Customer ID'],as_index=False).agg({'Date':lambda x:  pd.Timedelta(x.drop_duplicates().diff().std()).days})['Date'].fillna(0)

    
    
    LRMFP = pd.DataFrame(columns=['Customer ID','Length','Recency','Frequency','Monetary','Periodicity'])
    
    LRMFP['Customer ID'] = info['Customer ID']
    LRMFP['Length'] = (info['Date']['max']-info['Date']['min']).dt.days +1
    LRMFP['Frequency'] = info['Invoice']['count']
    LRMFP['Monetary'] = round(dataframe_in.groupby('Customer ID').sum()['Gains'].values/LRMFP['Frequency'],2)
    LRMFP['Recency'] = info2['Date']
    LRMFP['Periodicity'] = info3
    return LRMFP
    

In [4]:
df_retail_LRMFP = custom_features(df_retail)
df_retail_LRMFP

,Customer ID,Length,Recency,Frequency,Monetary,Periodicity
0,12346.0,197,342.0,11,33.90,43.0
1,12347.0,38,8.6,2,661.66,0.0
2,12348.0,1,14.8,1,222.16,0.0
3,12349.0,183,94.8,3,890.38,101.0
4,12351.0,1,2.2,1,300.93,0.0
...,...,...,...,...,...,...
4309,18283.0,277,193.0,6,103.23,61.0
4310,18284.0,1,13.4,1,461.68,0.0
4311,18285.0,1,59.2,1,427.00,0.0
4312,18286.0,248,94.2,2,648.22,0.0


**Resúesta de Pauta:**

## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [5]:
class MinMax(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X = (X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0))
        return X

In [8]:
ct = ColumnTransformer([("MinMax",MinMax(),[
                "Length",
                "Recency",
                "Frequency",
                "Monetary",
                "Periodicity"],)])

In [9]:
df_retail_LRMFP_minmax = pd.DataFrame(
    ct.fit_transform(df_retail_LRMFP),columns=ct.transformers_[0][2])

df_retail_LRMFP_minmax = pd.concat([df_retail_LRMFP['Customer ID'],df_retail_LRMFP_minmax],axis=1)
df_retail_LRMFP_minmax

,Customer ID,Length,Recency,Frequency,Monetary,Periodicity
0,12346.0,0.525469,0.922288,0.049020,0.002853,0.168627
1,12347.0,0.099196,0.022666,0.004902,0.055691,0.000000
2,12348.0,0.000000,0.039396,0.000000,0.018699,0.000000
3,12349.0,0.487936,0.255262,0.009804,0.074943,0.396078
4,12351.0,0.000000,0.005397,0.000000,0.025329,0.000000
...,...,...,...,...,...,...
4309,18283.0,0.739946,0.520237,0.024510,0.008689,0.239216
4310,18284.0,0.000000,0.035618,0.000000,0.038859,0.000000
4311,18285.0,0.000000,0.159201,0.000000,0.035940,0.000000
4312,18286.0,0.662198,0.253643,0.004902,0.054560,0.000000


### 1.3.2 `T-SNA` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo T-sne de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [15]:
pipe = Pipeline([('CustomFeatures',FunctionTransformer(custom_features)),
      ("MinMax",ct),
      ("TSNE",TSNE(n_components=2,random_state=1))])

df_retail_tsne = pipe.fit_transform(df_retail)
df_retail_tsne.shape


C:\Users\tacom\anaconda3\envs\datascience\lib\site-packages\sklearn\manifold\_t_sne.py:795: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

C:\Users\tacom\anaconda3\envs\datascience\lib\site-packages\sklearn\manifold\_t_sne.py:805: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



(4314, 2)

In [16]:
fig = px.scatter(df_retail_tsne,x=0,y=1,title='T-SNE para n=2 de LRMFP sobre df_retail')
fig.show()

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero permutando el algoritmo de reducción de dimensionalidad por k-means.

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [17]:
sse = []

clusters = list(range(1, 20))
for k in clusters:
    pipe = Pipeline([('CustomFeatures',FunctionTransformer(custom_features)),
      ("MinMax",ct),
      ("Kmeans",KMeans(n_clusters=k,random_state=1))])
    kmeans = pipe.fit(df_retail)[-1]
    sse.append(kmeans.inertia_)

fig = go.Figure(data=[
    go.Scatter(x = clusters, y=sse, mode='lines'),
    go.Scatter(x = clusters, y=sse, mode='markers')
])

fig.update_layout(title="Metodo del codo de 1 a 20 clusters",
                 xaxis_title = "Numero de Clusters",
                 yaxis_title = "SSE",
                 showlegend=False)
fig.show()

Se escoge k=6 puesto que no se presenta una gran diferencia de error SSE al seguir aumentando el número de clusters. Por otro lado, 6 tampoco es un número muy grande por lo que no el algoritmo no consume muchos recursos computacionales.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [20]:
pipe = Pipeline([('CustomFeatures',FunctionTransformer(custom_features)),
      ("MinMax",ct),
      ("Kmeans",KMeans(n_clusters=6,random_state=1))])

kmeans = pipe.fit(df_retail)[-1]

In [21]:
LRMFP_kmeans = df_retail_LRMFP.copy()
LRMFP_kmeans['Cluster'] = kmeans.labels_
LRMFP_kmeans = round(LRMFP_kmeans.groupby('Cluster').mean(),2)
LRMFP_kmeans

,Length,Recency,Frequency,Monetary,Periodicity
Cluster,,,,,
0,7.94,31.39,1.37,359.03,0.45
1,344.14,283.21,16.03,457.08,26.62
2,233.99,94.75,3.40,367.33,26.49
3,308.56,133.59,4.35,363.44,100.17
4,233.77,204.39,7.30,403.52,30.51
5,119.85,85.00,3.26,357.61,12.91


**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

Con respecto al resultado obtenido por nosotros, si es posible observar agrupaciones pues las combinaciones de valores de cada columna varían en función del cluster. Para los clusters se puede identificar este tipo de cliente. A continuación se realiza una explicación de que tipo de cliente corresponde a cada cluster.

- Cluster 0: Clientes recientes o nuevos, que tienen menor Recency y Periodicity que el resto de los clusters, por ende tienen mayor regularidad en sus visitas en un tiempo reciente.

- Cluster 1: Clientes más antiguos, pero con el mayor valor de Recency, es decir, visitan la tienda menos actualmente. Por otro lado, son los que más han gastado promedio.

- Cluster 2: Clientes que llevan harto tiempo comprando en la tienda (por gran valor de length), que en la actualidad son clientes esporádicos (por su valor de recency), pero que tienen igual visitan o vistaban la tienda frecuentemente. 

- Cluster 3: Clientes bien antiguos (dada su length), con una alta cantidad de visitas, pero con una frecuencia relativamente constante desde sus inicios hasta la actualidad.

- Cluster 4: Clientes antiguos, que tenían gran frecuencia de visitas en el pasado pero recientemente no. Adicionalmente, presentan un Monetary elevado, lo que indica que tienden a gastar más en la tienda.

- Cluster 5: Clientes que no son tan antiguos, pero tampoco recientes, con una buena regularidad en sus visitas.

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [22]:
fig = px.scatter(df_retail_tsne,x=0,y=1,color=kmeans.labels_,title='T-SNE para n=2 de LRMFP sobre df_retail, utilizando k=6 clusters de Kmeans')
fig.show()

Existen clusters muy bien identificados, tales como los casos de 5, 4, 3 y 1. Sin embargo, en el caso de los clusters 0 y 2, se presentan algunas agrupaciones de sus clusters respectivos separadas de la agrupación principal o separados mediante algún otro cluster (2 separado por 5).

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab** solo para tareas. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>